In [1]:
import sys
sys.path.append('..')  # allow this notebook to find equal-level directories
%pylab inline
from importing_modules import *
# pyomo.environ as oe, seaborn as sns, plotly.plotly as py, plotly.graph_objs as go
# from util.gjh_wrapper import gjh_solve, make_df, from vis import acres_bars, zL_bars

Populating the interactive namespace from numpy and matplotlib


## Create problem instance

In [2]:
# Load data for each set, parameter, etc. to define a problem instance
objwrapper = CostObj()
# lrsegs = ['N42071SL2_2410_2700']
lrsegs = ['N51133RL0_6450_0000']
data = objwrapper.load_data(savedata2file=False, lrsegs_list=lrsegs)

# ---- Set the tau target load ---- (but not for sediment)
for k in data.tau:
    print(k)
    if k[1]!='S':
        data.tau[k] = 12
        taustr = str(round(data.tau[k], 1))
    
# Create concrete problem instance using the separately defined optimization model
mdl = objwrapper.create_concrete(data=data)
# Print the target load reduction values
for l in mdl.LRSEGS:
    for p in mdl.PLTNTS:
        print('%s: %d' % (mdl.tau[l,p], mdl.tau[l,p].value))

# ---- Solver name ----
localsolver = True
solvername = 'ipopt'
# solvername = 'minos'

('N51133RL0_6450_0000', 'N')
('N51133RL0_6450_0000', 'P')
('N51133RL0_6450_0000', 'S')
tau[N51133RL0_6450_0000,P]: 12
tau[N51133RL0_6450_0000,S]: 5
tau[N51133RL0_6450_0000,N]: 12


## Solve problem instance

In [3]:
df_list = []
solution_objectives = OrderedDict()
for ii in range(1, 10):
    print(ii)
    
    # reinitialize the variables
    for k in mdl.x:
#         mdl.x[k] = float(random.randrange(0, 600001))/100
        mdl.x[k] = round(random.uniform(0, 6000), 2)
        
    # print the tau parameter
    for k in data.tau:
        print(mdl.tau[k].value)
        taustr = str(round(mdl.tau[k].value, 1))
        
    # Set names for saving output
    looptimestamp = datetime.now().strftime('%Y-%m-%d_%H%M%S')
    loopname = ''.join(['output/loadobj_startingpoint', str(ii),
                        '_tau', taustr, '_', solvername, '_',
                           looptimestamp])
        
    # Solve The Model
    myobj = SolveAndParse(instance=mdl, data=data, localsolver=localsolver, solvername=solvername)
    logfilename = os.path.join(projectpath, ''.join([loopname, '.txt']))
    # set filepath for saving information about all of the solver iterates
    myobj.modify_ipopt_options(optionsfilepath='ipopt.opt',
                               newoutputfilepath=os.path.join(projectpath, ''.join([loopname, '.iters'])))
    merged_df = myobj.solve(logfilename=logfilename)
    print('shape of df is: %s' % str(merged_df.shape))
    print('\nObjective is: %d' % oe.value(mdl.Total_Cost))
    
    # Save this run's objective value in a list
    solution_objectives[mdl.tau[k].value] = oe.value(mdl.Total_Cost)
    merged_df['solution_objectives'] = oe.value(mdl.Total_Cost)
    
    # Label this run in the dataframe
    merged_df['tau'] = mdl.tau[k].value
    merged_df['startpointiterate'] = ii
    sorteddf_byacres = merged_df.sort_values(by='acres')
    
#     # ---- Make zL Figure ----
#     savefilepathandname = os.path.join(projectpath, ''.join([loopname, '.png']))

#     zL_bars(df=merged_df, instance=mdl,
#             savefig=True, savefilepathandname=savefilepathandname)
    
    
#     # ---- Make Acres Figure ----
#     savefilepathandname = os.path.join(projectpath, ''.join([loopname, '.png']))

#     objstr = ''.join(['Objective is: ', str(round(mdl.Total_Cost(), 2))])
#     titlestr = '\n'.join([objstr, 'labels are (cost per unit, total bmp instance cost)'])

#     acres_bars(df=sorteddf_byacres, instance=mdl, titlestr=titlestr,
#                savefig=True, savefilepathandname=savefilepathandname)
    
#     # -- Gradient, Jacobian, Hessian
#     gjh_filename, g = gjh_solve(instance=mdl,
#                             keepfiles=True,
#                             amplenv=ampl,
#                             basegjhpath=os.getcwd())

#     g_df = make_df(instance=mdl, filterbydf=merged_df, g=g)

#     g_df = sorteddf_byacres.merge(g_df, how='left',
#                                   on=['bmpshortname', 'landriversegment', 'loadsource'],
#                                   sort=False)
    
#     # ---- Make gradient Figure ----
#     g_df_filtered = g_df
#     keystrs = [str([x, y]) for x, y in zip(g_df_filtered['bmpshortname'], g_df_filtered['loadsource'])]
#     # Make Figure
#     fig = plt.figure(figsize=(10, 4))
#     rects = plt.barh(y=keystrs, width=g_df_filtered['g'])
#     ax = plt.gca()

#     ax.set_position([0.3,0.1,0.5,0.8])

#     plt.savefig(os.path.join(projectpath, ''.join([loopname, '.png'])))
    # Save all of the solutions in a list
    df_list.append(sorteddf_byacres)
    
# Save the results to a .csv file
alldfs = pd.concat(df_list, ignore_index=True)
alldfs['x'] = list(zip(alldfs.bmpshortname, alldfs.landriversegment, alldfs.loadsource, alldfs.totalannualizedcostperunit))
filenamestr = ''.join(['output/costobj_difstartpts_alldfs', '_', solvername, '_',
                           datetime.now().strftime('%Y-%m-%d_%H%M%S'), '.csv'])
alldfs.to_csv(os.path.join(projectpath, filenamestr))

1
12
12
5
Solver log file: '/Users/Danny/Desktop/CATEGORIES/CAREER_MANAGEMENT/CRC_ResearchScientist_Optimization/Optimization_Tool/2_ExperimentFolder/ampl/OptEfficiencySubProblem/output/loadobj_startingpoint1_tau5_ipopt_2018-08-09_104800.txt'
Solver solution file: '/var/folders/qx/2v7d4gls4d10zzr_ptplrx240000gr/T/tmpfpjboqap.pyomo.sol'
Solver problem files: ('/var/folders/qx/2v7d4gls4d10zzr_ptplrx240000gr/T/tmpfpjboqap.pyomo.nl',)
Ipopt 3.12.8: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.8, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros 

  72  2.8423846e+06 6.60e-03 6.02e+05  -1.0 1.05e+02    -  1.00e+00 7.77e-01h  1
  73  2.8419348e+06 2.31e-03 5.08e+05  -1.0 2.34e+01    -  1.00e+00 3.28e-01h  1
  74  2.8411628e+06 2.80e-04 7.44e+04  -1.0 1.58e+01    -  1.00e+00 8.56e-01h  1
  75  2.8411306e+06 2.49e-04 5.59e+04  -1.0 2.27e+00    -  1.00e+00 2.51e-01h  1
  76  2.8410468e+06 2.96e-05 1.72e+04  -1.0 1.70e+00    -  1.00e+00 8.59e-01h  1
  77  2.8350366e+06 1.32e-05 1.05e+06  -1.7 3.93e+01    -  4.91e-01 6.22e-01f  1
  78  2.8314659e+06 1.75e-06 2.47e+06  -1.7 1.49e+01    -  2.77e-01 8.43e-01f  1
  79  2.8314068e+06 1.61e-06 6.33e+05  -1.7 2.34e+00    -  9.83e-01 8.50e-02f  2
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  80  2.8309452e+06 6.97e-07 3.99e+05  -1.7 3.43e+00    -  1.00e+00 6.35e-01h  1
  81  2.8307294e+06 0.00e+00 2.07e-07  -1.7 9.09e-01    -  1.00e+00 1.00e+00h  1
  82  2.8278755e+06 0.00e+00 1.51e+06  -8.6 9.77e+00    -  2.44e-01 1.00e+00f  1
  83  2.8277565e+06 0.00e+00

  29  4.1318477e+07 5.35e+04 7.29e+02  -1.0 1.45e+05    -  2.18e-02 6.30e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  30  4.1285745e+07 5.34e+04 3.26e+02  -1.0 7.82e+04    -  8.08e-02 2.71e-03f  1
  31  4.0215853e+07 5.07e+04 6.03e+02  -1.0 1.32e+05    -  2.41e-02 5.08e-02f  1
  32  3.9278220e+07 4.93e+04 1.03e+03  -1.0 1.20e+05    -  1.26e-02 2.79e-02f  1
  33  4.0463886e+07 4.69e+04 1.09e+03  -1.0 1.73e+05    -  2.15e-03 4.86e-02h  1
  34  4.0322518e+07 4.62e+04 1.36e+03  -1.0 1.29e+05    -  5.87e-04 1.38e-02f  1
  35  4.1463294e+07 4.38e+04 1.27e+03  -1.0 1.19e+05    -  1.32e-03 5.25e-02h  1
  36  4.1735901e+07 4.21e+04 1.22e+03  -1.0 8.07e+04    -  4.13e-02 3.84e-02h  1
  37  4.1760590e+07 3.99e+04 1.00e+03  -1.0 6.89e+04    -  9.05e-04 5.22e-02h  1
  38  4.1735264e+07 3.97e+04 1.09e+03  -1.0 5.81e+04    -  3.19e-02 6.59e-03f  1
  39  4.1735250e+07 3.97e+04 1.09e+03  -1.0 6.71e+04    -  6.57e-04 5.70e-05h  1
iter    objective    inf_pr 

 123  4.1357474e+06 9.97e+01 6.20e+02  -1.0 9.81e+02    -  5.39e-02 7.65e-01f  1
 124  4.0548517e+06 8.64e+01 6.15e+02  -1.0 5.74e+03    -  3.47e-02 1.33e-01f  1
 125  3.7454002e+06 3.84e+01 5.83e+02  -1.0 4.49e+03    -  1.36e-01 5.54e-01f  1
 126  3.5673969e+06 2.60e+01 5.73e+02  -1.0 3.11e+03    -  2.65e-02 3.24e-01f  1
 127  3.2643695e+06 1.17e+01 6.72e+02  -1.0 2.20e+03    -  4.79e-02 5.53e-01f  1
 128  3.0969247e+06 9.15e+00 5.97e+02  -1.0 6.65e+03    -  1.73e-01 2.22e-01f  1
 129  3.0259259e+06 4.88e+00 4.30e+02  -1.0 5.22e+02    -  2.86e-01 4.71e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 130  2.9288601e+06 1.14e+00 3.49e+02  -1.0 8.05e+02    -  1.96e-01 7.66e-01f  1
 131  2.9047619e+06 6.31e-01 2.21e+02  -1.0 6.97e+02    -  3.66e-01 4.40e-01f  1
 132  2.8870339e+06 3.99e-01 7.32e+02  -1.0 7.25e+02    -  6.32e-02 3.66e-01f  1
 133  2.8784644e+06 2.99e-01 4.26e+02  -1.0 5.18e+02    -  5.48e-01 2.45e-01f  1
 134  2.8633370e+06 1.58e-01

   9  5.7849828e+08 2.49e+05 6.54e+01  -1.0 1.50e+05    -  5.33e-04 4.95e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10  4.8321132e+08 2.37e+05 6.66e+01  -1.0 1.82e+05    -  1.90e-03 4.95e-02f  1
  11  3.9632018e+08 2.25e+05 6.75e+01  -1.0 1.76e+05    -  5.52e-03 5.13e-02f  1
  12  3.1978149e+08 2.12e+05 6.85e+01  -1.0 1.25e+05    -  4.62e-03 5.65e-02f  1
  13  2.6991921e+08 1.99e+05 6.75e+01  -1.0 1.02e+05    -  2.79e-02 6.08e-02f  1
  14  2.2287552e+08 1.82e+05 6.94e+01  -1.0 1.34e+05    -  1.43e-03 8.67e-02f  1
  15  1.8826634e+08 1.66e+05 6.83e+01  -1.0 7.28e+04    -  3.22e-02 8.61e-02f  1
  16  1.5708638e+08 1.47e+05 7.00e+01  -1.0 7.57e+04    -  6.68e-03 1.15e-01f  1
  17  1.2156902e+08 1.29e+05 7.17e+01  -1.0 2.13e+05    -  5.62e-03 1.23e-01f  1
  18  9.7546123e+07 1.13e+05 7.18e+01  -1.0 8.55e+04    -  2.14e-02 1.28e-01f  1
  19  7.4865793e+07 9.61e+04 7.24e+01  -1.0 5.33e+04    -  1.62e-02 1.46e-01f  1
iter    objective    inf_pr 

...

  22  7.5571748e+07 9.49e+04 6.95e+01  -1.0 6.97e+04    -  5.06e-03 7.61e-03f  1
  23  7.4303157e+07 9.38e+04 6.85e+01  -1.0 7.11e+04    -  1.43e-02 1.09e-02f  1
  24  6.2758711e+07 8.40e+04 6.65e+01  -1.0 6.15e+04    -  3.89e-02 1.05e-01f  1
  25  6.0720317e+07 8.19e+04 6.62e+01  -1.0 2.22e+04    -  8.09e-03 2.51e-02f  1
  26  5.7012036e+07 7.80e+04 7.15e+01  -1.0 1.93e+04    -  2.04e-02 4.78e-02f  1
  27  5.4557876e+07 7.52e+04 6.20e+01  -1.0 1.73e+04    -  4.62e-02 3.50e-02f  1
  28  5.2061934e+07 7.23e+04 3.57e+02  -1.0 1.70e+04    -  1.25e-01 3.92e-02f  1
  29  4.7539245e+07 6.39e+04 3.84e+02  -1.0 6.47e+04    -  9.72e-04 1.16e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  30  4.6732570e+07 5.86e+04 4.20e+02  -1.0 1.19e+05    -  5.18e-02 8.32e-02f  1
  31  4.6012043e+07 5.74e+04 5.25e+01  -1.0 2.93e+04    -  4.78e-02 1.97e-02f  1
  32  4.4322172e+07 5.52e+04 8.61e+02  -1.0 1.74e+04    -  2.81e-04 3.92e-02f  1
  33  4.5003819e+07 5.17e+04

 117  2.8412220e+06 2.32e-03 3.99e+05  -1.0 3.42e+00    -  1.00e+00 4.75e-01h  1
 118  2.8410893e+06 6.32e-04 1.22e+05  -1.0 1.80e+00    -  1.00e+00 6.99e-01h  1
 119  2.8410638e+06 3.78e-04 6.60e+04  -1.0 5.41e-01    -  1.00e+00 4.59e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 120  2.8410421e+06 1.01e-04 2.19e+04  -1.0 2.92e-01    -  1.00e+00 7.04e-01h  1
 121  2.8410380e+06 6.08e-05 1.12e+05  -1.0 8.65e-02    -  1.00e+00 4.57e-01h  1
 122  2.8410345e+06 1.60e-05 1.29e+05  -1.0 4.70e-02    -  1.00e+00 7.07e-01h  1
 123  2.8410338e+06 9.64e-06 6.11e+05  -1.0 1.38e-02    -  1.00e+00 4.59e-01h  1
 124  2.8410333e+06 2.40e-06 5.98e+05  -1.0 7.44e-03    -  1.00e+00 7.19e-01h  1
 125  2.8410332e+06 1.42e-06 1.89e+06  -1.0 2.09e-03    -  1.00e+00 4.78e-01h  1
 126  2.8410331e+06 2.34e-07 8.76e+05  -1.0 1.09e-03    -  1.00e+00 7.91e-01h  1
 127  2.8410331e+06 1.32e-07 1.88e+06  -1.0 2.28e-04    -  1.00e+00 5.49e-01h  1
 128  2.8410331e+06 0.00e+00

  21  7.3226689e+07 9.67e+04 6.65e+01  -1.0 2.28e+04    -  7.29e-02 6.48e-03f  1
  22  6.9312406e+07 9.25e+04 6.68e+01  -1.0 2.27e+04    -  3.21e-03 4.35e-02f  1
  23  6.3836197e+07 8.64e+04 6.52e+01  -1.0 2.17e+04    -  2.91e-02 6.54e-02f  1
  24  6.0312653e+07 8.24e+04 6.67e+01  -1.0 2.03e+04    -  8.11e-03 4.69e-02f  1
  25  5.7680150e+07 7.92e+04 1.16e+02  -1.0 1.94e+04    -  1.39e-02 3.82e-02f  1
  26  5.4595855e+07 7.54e+04 2.05e+02  -1.0 1.86e+04    -  1.14e-03 4.81e-02f  1
  27  5.3977591e+07 7.46e+04 2.10e+02  -1.0 1.77e+04    -  6.33e-02 1.06e-02f  1
  28  4.8374224e+07 6.29e+04 4.08e+02  -1.0 6.56e+04    -  1.83e-03 1.57e-01f  1
  29  4.5983049e+07 5.77e+04 1.92e+02  -1.0 5.44e+04    -  1.64e-01 8.34e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  30  4.5096507e+07 5.64e+04 4.89e+01  -1.0 1.36e+04    -  3.50e-02 2.23e-02f  1
  31  4.2932399e+07 5.06e+04 8.45e+02  -1.0 7.05e+04    -  8.50e-04 1.03e-01f  1
  32  4.1424693e+07 4.63e+04

Ipopt 3.12.8: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.8, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:     3609
Number of nonzeros in Lagrangian Hessian.............:     5575

Total number of variables............................:     1339
                     variables with only lower bounds:     1339
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total

  77  2.9109283e+06 3.18e+00 3.91e+04  -1.0 3.18e+02    -  2.22e-01 1.08e-01f  1
  78  2.9037398e+06 2.78e+00 6.85e+04  -1.0 2.81e+02    -  8.32e-02 1.25e-01f  1
  79  2.8745284e+06 1.22e+00 4.44e+05  -1.0 2.45e+02    -  1.04e-01 5.53e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  80  2.8646158e+06 7.91e-01 1.17e+05  -1.0 1.16e+02    -  4.33e-01 3.35e-01f  1
  81  2.8572583e+06 5.11e-01 3.93e+05  -1.0 8.86e+01    -  5.45e-01 3.41e-01f  1
  82  2.8506891e+06 2.83e-01 2.51e+05  -1.0 1.37e+02    -  4.51e-01 4.39e-01f  1
  83  2.8490797e+06 2.32e-01 8.58e+05  -1.0 1.07e+02    -  3.85e-01 1.77e-01f  2
  84  2.8436212e+06 6.40e-02 1.07e+06  -1.0 1.32e+02    -  9.93e-01 7.00e-01h  1
  85  2.8425772e+06 3.53e-02 1.80e+06  -1.0 3.19e+01    -  1.00e+00 3.99e-01h  1
  86  2.8415133e+06 5.49e-03 6.88e+05  -1.0 1.19e+01    -  1.00e+00 6.92e-01h  1
  87  2.8412859e+06 1.81e-03 3.98e+05  -1.0 3.68e+00    -  1.00e+00 4.71e-01h  1
  88  2.8411099e+06 4.92e-04

In [4]:
# solver = oe.SolverFactory('ipopt')
# results = solver.solve(mdl, tee=True, symbolic_solver_labels=True,
#                                    keepfiles=False)

In [5]:
i=0
for v in mdl.component_data_objects(oe.Var):
    if v.stale:
        continue
    i+=1
#     print(str(v), ' = ', value(v))
    
print(i)

1339


In [6]:
# for k in mdl.x.keys():
#     print(mdl.x[k].stale)

In [7]:
from util.solution_wrangler import get_nonzero_var_names_and_values
nzvnames, nzvvalues = get_nonzero_var_names_and_values(mdl, onlynotstale=True)

In [8]:
print(len(nzvnames))
nzvnames[1]

22


'x[UrbanNMPlanHR,N51133RL0_6450_0000,ntg]'

In [9]:
from util.solution_wrangler import get_nonzero_var_df
nonzerodf = get_nonzero_var_df(mdl, addcosttbldata=data.costsubtbl)

In [10]:
print(nonzerodf.shape)
nonzerodf.head(5)

(22, 6)


,acres,bmpshortname,landriversegment,loadsource,totalannualizedcostperunit,totalinstancecost
0,423.264714,DitchFilter,N51133RL0_6450_0000,pas,10.05,4253.810373
1,2834.604032,UrbanNMPlanHR,N51133RL0_6450_0000,ntg,0.00,0.000000
2,32.488785,UrbanNMPlanHR,N51133RL0_6450_0000,mcn,0.00,0.000000
3,2023.723409,UrbanNMPlanHR,N51133RL0_6450_0000,nch,0.00,0.000000
4,768.559607,BioSwale,N51133RL0_6450_0000,ntg,989.19,760251.478135


In [11]:
oe.value(oe.value(mdl.Total_Cost))

2827702.934382514

In [12]:
# # Other ways to access the optimal values:
# mdl.x['HRTill', 'N51133RL0_6450_0000', 'oac'].value
#
# tol = 1e-6
# for b in mdl.BMPS:
#     for lmbda in mdl.LOADSRCS:
#         bval = mdl.x[b, 'N51133RL0_6450_0000', lmbda].value
#         if not not bval:
#             if abs(bval)>tol:
#                 print('(%s, %s): %d' % (b, lmbda, bval))